In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
cmap = matplotlib.cm.get_cmap('Spectral')
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.metrics import confusion_matrix

# Clasificacion

Vamos a hacer una introduccion a problemas de clasificacion adelantandonos nuevamente. Veamos como seria resolver un problema de clasificacion ingenuo con ayuda de sklearn en datos reales.

# El problema:

En esta cuarentena, estoy muy metido en la NBA. Con mi grupo de adictos, empezamos a charlar para matar el tiempo y me surgio una pregunta. ~Puedo hablar de basquet en vez de trabajar?~ Puedo mentir con seguridad sobre las estadisticas que leo en los articulos?

Para eso, me puse a explorar un poco. En primer lugar, necesito datos. Para eso, recurro a una pagina llamada basketballreference.com donde se almacenan todos los datos posibles. Para hacer las cosas mas simples, me voy a limitar a una unica temporada, en la que los roles posicionales son mas o menos los mismos en la liga (salvo en Houston) y no tengo jugadores repetidos (salvo los que fueron transferidos, pero ahora mismo no me interesa ser demasiado cauto en el procesado).

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
BeautifulSoup?

In [ ]:
url = "https://www.basketball-reference.com/leagues/NBA_{}_advanced.html".format(2016)# this is the HTML from the given URL
html = urlopen(url)
soup = BeautifulSoup(html)

In [ ]:
soup.findAll('tr', limit=2)# use getText()to extract the text we need into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis

In [ ]:
headers = headers[1:]
rows = soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
stats = pd.DataFrame(player_stats, columns = headers)
stats=stats.drop('\xa0',axis=1)

In [ ]:
stats.head()

In [ ]:
stats[stats['Tm']=='TOT']['Player']

En particular, todo a partir de OWS son estadisticas que me hablan de "cuan bueno es un jugador", y estan obtenidas con ciertas combinaciones de las previas.

En particular, yo me quiero plantear el siguiente problema. Que tan bien tienen que jugar los jugadores para clasificar a Playoffs? 

In [ ]:
teams=stats['Tm'].unique()
teams

In [ ]:
no_playoff_west=np.asarray(['SAC','DEN','LAL','MIN','PHO','UTA','NOP'])
print(len(no_playoff_west))
playoff_west=np.asarray(['GSW','SAS','OKC','LAC','POR','DAL','MEM','HOU'])
print(len(playoff_west))

no_playoff_east=np.asarray(['CHI','WAS','ORL','MIL','NYK','BRK','PHI'])
print(len(no_playoff_east))
playoff_east=np.asarray(['CLE','TOR','MIA','ATL','BOS','CHO','IND','DET'])
print(len(playoff_east))

In [ ]:
stats['Pos'].value_counts()

# El procesado

En principio, yo tengo mi dataframe y ahora lo unico que voy a hacer es separar en Train y Test y luego asignarles las clases. En particular, voy considerar jugadores que jugaron cierta cantidad de los partidos al menos cierta cantidad de minutos y que participaron mucho del partido.

In [ ]:
stats_now=stats.copy()
stats_now=stats_now[stats_now['Pos'].isin(['SG','PF','PG','C','SF'])]
stats_now["G"]=pd.to_numeric(stats_now["G"])
stats_now["MP"]=pd.to_numeric(stats_now["MP"])
stats_now["USG%"]=pd.to_numeric(stats_now["USG%"])
#stats_now=stats_now[stats_now["USG%"]>stats_now["USG%"].mean()]
stats_now["MPperG"]=stats_now["MP"]/stats_now["G"]
#stats_now=stats_now[stats_now["G"]>=40]
#stats_now=stats_now[stats_now["MPperG"]>=25]
stats_now=stats_now[stats_now["Tm"]!="TOT"]


In [ ]:
stats_now=stats_now.replace(['PG','SG','SF','PF','C'],[1,2,3,4,5])
stats_now['Pos'].value_counts()

Generemos los labels. Voy a aplicar el siguiente codigo:

0: no playoff oeste
1: playoff oeste
2: no playoff este
3: playoff este

In [ ]:
stats_now['label']=stats_now["Tm"]
stats_now['label']=stats_now['label'].replace(no_playoff_west,0)
stats_now['label']=stats_now['label'].replace(playoff_west,1)
stats_now['label']=stats_now['label'].replace(no_playoff_east,2)
stats_now['label']=stats_now['label'].replace(playoff_east,3)
print(stats_now['label'].value_counts())
#print(stats_now['Tm'].value_counts())

In [ ]:
stats_now.info()

Bien, ahora definamos las caracteristicas que voy a utilizar. Para visualizar facilmente voy a elegir unicamente dos: PER y USG%. Guardo ademas el nombre, la posicion y el label.

In [ ]:
stats_now=stats_now[['Player','Pos','PER','USG%','label']]
stats_now['PER']=pd.to_numeric(stats_now['PER'])
stats_now['USG%']=pd.to_numeric(stats_now['USG%'])

In [ ]:
print(stats_now.iloc[np.where(stats_now["Player"]=="LeBron James")])
print(stats_now.iloc[np.where(stats_now["Player"]=="Will Barton")])

Veamos si hay jugadores repetidos:

In [ ]:
print(stats_now['Player'].value_counts())
plt.hist(stats_now['Player'].value_counts())

Los hay pero son pocos por lo que no me voy a gastar en corregirlo.


Dividamos en train y test. Como cada posicion es un mundo, voy a estratificar para que train y test tengan mismas proporciones de cada posicio



In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=445543)
for train_index, test_index in split.split(stats_now, stats_now["Pos"]):
    strat_train_set = stats_now.iloc[train_index]
    strat_test_set = stats_now.iloc[test_index]

In [ ]:
stats_train = strat_train_set.drop(["Player","Pos","label"], axis=1) # drop labels for training set
stats_train_labels = strat_train_set["label"].copy()
stats_test = strat_test_set.drop(["Player","Pos","label"], axis=1) # drop labels for training set
stats_test_labels = strat_test_set["label"].copy()


Veamos cuantos datos tengo para interpolar:

In [ ]:
print(len(stats_train))

In [ ]:
plt.hist(stats_train['PER'],bins=10)
plt.xlabel(r'PER')
plt.show()
plt.hist(stats_train['USG%'],bins=10)
plt.xlabel(r'USG%')
plt.show()
plt.scatter(stats_train['PER'][stats_train_labels==0],stats_train['USG%'][stats_train_labels==0], c='red', label="No Playoff Oeste")
plt.scatter(stats_train['PER'][stats_train_labels==1],stats_train['USG%'][stats_train_labels==1], c='blue', label="Playoff Oeste")
plt.legend(loc='upper left',framealpha =0.1)
plt.xlim(9.0,31.0)
plt.ylim(15.0,35.0)
plt.xlabel(r'PER')
plt.ylabel('USG%')
plt.show()
plt.scatter(stats_train['PER'][stats_train_labels==2],stats_train['USG%'][stats_train_labels==2], c='orange', label="No Playoff Este")
plt.scatter(stats_train['PER'][stats_train_labels==3],stats_train['USG%'][stats_train_labels==3], c='green', label="Playoff Este")
plt.legend(loc='upper left',framealpha =0.1)
plt.xlabel(r'PER')
plt.ylabel('USG%')
plt.xlim(9.0,31.0)
plt.ylim(15.0,35.0)

En el Oeste, esperaria tener muy pocos falsos positivos pero una gran cantidad de falsos negativos. En el Este pasa lo mismo pero menos pronunciado.

In [ ]:
print(strat_train_set[strat_train_set["PER"]<12.0])
print(strat_train_set[strat_train_set["PER"]>25.0])

#Clasificacion

Entrenemos un clasificador, para Este y Oeste separados.

In [ ]:
X_este=np.asarray(strat_train_set[(stats_train_labels==2) | (stats_train_labels==3)][["PER","USG%"]])
print(X_este.shape)
y_este=np.asarray(strat_train_set[(stats_train_labels==2) | (stats_train_labels==3)][["label"]])
y_este=np.where(y_este==2,0,1)[:,0]
print(y_este.shape)

X_oeste=np.asarray(strat_train_set[(stats_train_labels==0) | (stats_train_labels==1)][["PER","USG%"]])
print(X_oeste.shape)
y_oeste=np.asarray(strat_train_set[(stats_train_labels==0) | (stats_train_labels==1)][["label"]])
print(y_oeste.shape)

El primer algoritmo importante que vamos a ver es Discriminante Lineal de Fisher. El objetivo del algoritmo, que es ademas un ejemplo de reduccion dimensional, es encontrar las componentes que minimizen la varianza intraclase y maximizen la varianza entre clases.

Para el caso de 2 clases, el discriminante lineal de fisher busca encontrar los coeficientes $w$ tales que la funcion discriminante es

$y=w_{0}+\vec{w}^{T}\cdot \vec{x}$

Y la superficie de decision usual es $y=0$. $w_{0}$ es el `intercept_` y $\vec{w}$ el vector de `coef_`.


In [ ]:
LDA_este=LinearDiscriminantAnalysis(solver='eigen')
LDA_este.fit(X_este,y_este)
print(LDA_este.intercept_,LDA_este.coef_)

Grafiquemos un poco los valores posibles

In [ ]:
x=np.linspace(9.0,31.0,100)
y=np.linspace(15.0,35.0,100)
Xtoplot,Ytoplot=np.meshgrid(x,y)
plt.xlim(9.0,31.0)
plt.ylim(15.0,35.0)
Z=LDA_este.intercept_ + LDA_este.coef_[0,0]*Xtoplot+LDA_este.coef_[0,1]*Ytoplot
plt.contourf(Xtoplot,Ytoplot,Z,levels=[-2.0,-1.0,0.0,1.0,2.0,2.5],alpha=0.6)
plt.colorbar()
plt.scatter(stats_train['PER'][stats_train_labels==2],stats_train['USG%'][stats_train_labels==2], c='orange', label="No Playoff Este")
plt.scatter(stats_train['PER'][stats_train_labels==3],stats_train['USG%'][stats_train_labels==3], c='green', label="Playoff Este")
plt.legend(loc='upper left',framealpha =0.1)
plt.xlabel(r'PER')
plt.ylabel('USG%')

Otra manera de obtener la funcion de decision es usando... `decision_function`

In [ ]:
x=np.linspace(9.0,31.0,100)
y=np.linspace(15.0,35.0,100)
Xtoplot,Ytoplot=np.meshgrid(x,y)
plt.xlim(9.0,31.0)
plt.ylim(15.0,35.0)
Z=LDA_este.decision_function(np.c_[Xtoplot.ravel(), Ytoplot.ravel()]).reshape(Xtoplot.shape)
plt.contourf(Xtoplot,Ytoplot,Z,levels=[-2.0,-1.0,0.0,1.0,2.0,2.5],alpha=0.6)
plt.colorbar()
plt.scatter(stats_train['PER'][stats_train_labels==2],stats_train['USG%'][stats_train_labels==2], c='orange', label="No Playoff Este")
plt.scatter(stats_train['PER'][stats_train_labels==3],stats_train['USG%'][stats_train_labels==3], c='green', label="Playoff Este")
plt.legend(loc='upper left',framealpha =0.1)
plt.xlabel(r'PER')
plt.ylabel('USG%')

In [ ]:
LDA_este.decision_function(X_este[0,:].reshape(1,-1))

Podemos asignar las clases utilizando la funcion de decision. sklearn nos lo provee con `predict` donde el umbral esta en $y=0$.

In [ ]:
x=np.linspace(9.0,31.0,100)
y=np.linspace(15.0,35.0,100)
Xtoplot,Ytoplot=np.meshgrid(x,y)
plt.xlim(9.0,31.0)
plt.ylim(15.0,35.0)
Z=LDA_este.predict(np.c_[Xtoplot.ravel(), Ytoplot.ravel()]).reshape(Xtoplot.shape)
plt.contourf(Xtoplot,Ytoplot,Z,levels=[0.0,0.5,1.0],colors=['orange','green'],alpha=0.6)
plt.scatter(stats_train['PER'][stats_train_labels==2],stats_train['USG%'][stats_train_labels==2], c='orange', label="No Playoff Este")
plt.scatter(stats_train['PER'][stats_train_labels==3],stats_train['USG%'][stats_train_labels==3], c='green', label="Playoff Este")
plt.legend(loc='upper left',framealpha =0.1)
plt.xlabel(r'PER')
plt.ylabel('USG%')

Una vez asignamos las clases, podemos calcular matriz de confusion y con eso distintas metricas

In [ ]:
cf_este=confusion_matrix(y_este,LDA_este.predict(X_este))
print(cf_este)
tn, fp, fn, tp = cf_este.ravel()
print("Sensitividad (TPR/Recall):%8.3f" % (tp/(tp+fn)))
print("Especificidad:%8.3f" % (tn/(tn+fp)))
print("Precision:%8.3f" % (tp/(tp+fp)))
print("FPR:%8.3f" % (fp/(tn+fp)))

De todas maneras, no tenemos demasiados datos pero podemos ver que lo mas alto es la especificidad. Es decir, no solemos asignar como positivos a los verdaderos negativos. Lo peor que tenemos es la sensitividad, que nos dice cuantos positivos nos perdemos. La precision nos dice cuan seguros podemos estar de un positivo.

El segundo algoritmo es el Perceptron Multicapa, que tambien obtiene una funcion discriminante. 

$y=f(\vec{w}^{T}\cdot\vec{\phi}(\vec{x}))$

Con $f(a)=\frac{a}{|a|}a$ y por convencion $\phi_0(\vec{x})=1$.

El Perceptron se resuelve con algoritmo iterativo que solo tiene asegurada la convergencia para problemas linealmente separables, que no es el caso. Sin embargo, ya que estamos probemoslo.

In [ ]:
Percep_este=LinearDiscriminantAnalysis(solver='eigen')
Percep_este.fit(X_este,y_este)
print(Percep_este.intercept_,Percep_este.coef_)

Como es un algoritmo que busca la funcion discriminante, se puede hacer lo mismo que hicimos para LDA.

El tercero es el de Regresion Logistica. A diferencia de los anteriores, este es un algoritmo que, bajo ciertas hipotesis, busca recuperar el posterior de la clase. Es decir, obtiene una probabilidad para cada clase. 

$y_{k}(\vec{w},\vec{x})=p(k|\vec{w},\vec{x})$


Es un ejemplo de un algoritmo discriminativo. En principio, se necesitan tantas funciones $y$ como clases. Sin embargo, el caso de 2 clases provee una simplificacion ya que como son probabilidades, $y_{0}+y_{1} = 1$. Entonces, llamo $y$ a $y_{1}$ y puedo escribir de manera compacta.

$t=0,1$

$p(t|\vec{x},\vec{w})=y(\vec{x},\vec{w})^{t}(1-y(\vec{x},\vec{w}))^{1-t}$

$y(\vec{w},\vec{x})=\sigma(\vec{w}^{T}\cdot\vec{\phi}(\vec{x}))$

Donde $\sigma$ es la funcion sigmoide.

Ahora la superficie de decision estara, para dos clases, en la recta de equiprobabilidad 

$p(0|\vec{w},\vec{x})=p(1|\vec{w},\vec{x})$

$1-y=y$

$y=0.5$

El error que se minimiza en este caso no es el de cuadrados minimos sino que es el de la entropia cruzada (_cross-entropy_):


$E(\vec{w})=-\sum_{n=1}^{N}(t_{n}\text{ln}(y_{n})+(1-t_{n})\text{ln}(1-y_{n}))$

Ya no se puede minimizar esto analiticamente pero se puede hacer de manera numerica.

In [ ]:
logistic_este=LogisticRegression()
logistic_este.fit(X_este,y_este)

Ahora, ademas de funcion de decision, tenemos probabilidaes:



In [ ]:
proba=logistic_este.predict_proba(X_este)
print(proba.shape)
print(proba[0],y_este[0])

Y tambien tenemos la opcion predict, tal como antes:

In [ ]:
x=np.linspace(9.0,31.0,100)
y=np.linspace(15.0,35.0,100)
Xtoplot,Ytoplot=np.meshgrid(x,y)
plt.xlim(9.0,31.0)
plt.ylim(15.0,35.0)
Z=logistic_este.predict(np.c_[Xtoplot.ravel(), Ytoplot.ravel()]).reshape(Xtoplot.shape)
plt.contourf(Xtoplot,Ytoplot,Z,levels=[0.0,0.5,1.0],colors=['orange','green'],alpha=0.6)
plt.scatter(stats_train['PER'][stats_train_labels==2],stats_train['USG%'][stats_train_labels==2], c='orange', label="No Playoff Este")
plt.scatter(stats_train['PER'][stats_train_labels==3],stats_train['USG%'][stats_train_labels==3], c='green', label="Playoff Este")
plt.legend(loc='upper left',framealpha =0.1)
plt.xlabel(r'PER')
plt.ylabel('USG%')

Podemos sacar mas informacion viendo la matriz de confusion

In [ ]:
from sklearn.metrics import confusion_matrix
cf_este=confusion_matrix(y_este,logistic_este.predict(X_este))
print(cf_este)
tn, fp, fn, tp = cf_este.ravel()
print("Sensitividad (TPR/Recall):%8.3f" % (tp/(tp+fn)))
print("Especificidad:%8.3f" % (tn/(tn+fp)))
print("Precision:%8.3f" % (tp/(tp+fp)))
print("FPR:%8.3f" % (fp/(tn+fp)))

Ya que tenemos probabilidades, yo me siento comodo jugando un poco con los umbrales de decision:

In [ ]:
x=np.linspace(9.0,31.0,100)
y=np.linspace(15.0,35.0,100)
Xtoplot,Ytoplot=np.meshgrid(x,y)
plt.xlim(9.0,31.0)
plt.ylim(15.0,35.0)
Z=logistic_este.predict_proba(np.c_[Xtoplot.ravel(), Ytoplot.ravel()])[:,1].reshape(Xtoplot.shape)
plt.contourf(Xtoplot,Ytoplot,Z,levels=[0.0,0.2,0.4,0.5,0.6,0.8,1.0],alpha=0.6)
plt.colorbar()
plt.scatter(stats_train['PER'][stats_train_labels==2],stats_train['USG%'][stats_train_labels==2], c='orange', label="No Playoff Este")
plt.scatter(stats_train['PER'][stats_train_labels==3],stats_train['USG%'][stats_train_labels==3], c='green', label="Playoff Este")
plt.legend(loc='upper left',framealpha =0.1)
plt.xlabel(r'PER')
plt.ylabel('USG%')

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_este, logistic_este.predict_proba(X_este)[:,1])

plt.figure(figsize=(8, 6))                         # Not shown
plt.plot(fpr, tpr, linewidth=2, label=None)
plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal
plt.axis([0, 1, 0, 1])
plt.scatter(fpr[np.argmin(np.abs(thresholds-0.5))],tpr[np.argmin(np.abs(thresholds-0.5))],color='red')                                    # Not shown in the book
plt.xlabel('False Positive Rate (Fall-Out)', fontsize=16) # Not shown
plt.ylabel('True Positive Rate (Recall)', fontsize=16)    # Not shown
plt.grid(True)                                            # Not shown
plt.show()

## Y si quiero mas inputs?

Bueno, en principio puedo elegir mas inputs

In [ ]:
stats_now=stats_now[['Player','Pos','PER','USG%','TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','label']]
for e in ['PER','USG%','TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%']:
  stats_now[e]=pd.to_numeric(stats_now[e])

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=445543)
for train_index, test_index in split.split(stats_now, stats_now["Pos"]):
    strat_train_set = stats_now.iloc[train_index]
    strat_test_set = stats_now.iloc[test_index]

In [ ]:
stats_train = strat_train_set.drop(["Player","Pos","label"], axis=1) # drop labels for training set
stats_train_labels = strat_train_set["label"].copy()
stats_test = strat_test_set.drop(["Player","Pos","label"], axis=1) # drop labels for training set
stats_test_labels = strat_test_set["label"].copy()

In [ ]:
X_este=np.asarray(stats_train[(stats_train_labels==2) | (stats_train_labels==3)])
print(X_este.shape)
y_este=np.asarray(strat_train_set[(stats_train_labels==2) | (stats_train_labels==3)][["label"]])
y_este=np.where(y_este==2,0,1)[:,0]
print(y_este.shape)

X_oeste=np.asarray(stats_train[(stats_train_labels==0) | (stats_train_labels==1)])
print(X_oeste.shape)
y_oeste=np.asarray(strat_train_set[(stats_train_labels==0) | (stats_train_labels==1)][["label"]])
print(y_oeste.shape)

In [ ]:
scaler=StandardScaler()
X_este_tr=scaler.fit_transform(X_este)
LDA_este_multi=LinearDiscriminantAnalysis(solver='eigen')
LDA_este_multi.fit(X_este_tr,y_este)

In [ ]:
from sklearn.metrics import confusion_matrix
cf_este=confusion_matrix(y_este,LDA_este_multi.predict(X_este_tr))
print(cf_este)
tn, fp, fn, tp = cf_este.ravel()
print("Sensitividad:%8.3f" % (tp/(tp+fn)))
print("Especificidad:%8.3f" % (tn/(tn+fp)))
print("Precision:%8.3f" % (tp/(tp+fp)))

Funciona mejor! Pero para plottear, hay que marginalizar en las otras variables.

Una alternativa es utilizar herramientas de reduccion de dimensionalidad. LDA es una de ellas, que baja de 12 features a 1.

In [ ]:
LDA_este_multi.decision_function(X_este_tr).shape
plt.hist(LDA_este_multi.decision_function(X_este_tr[y_este==0]),color='orange',histtype='step',label='No playoff este')
plt.hist(LDA_este_multi.decision_function(X_este_tr[y_este==1]),color='green',histtype='step',label='Playoff este')
plt.axvline(x=0,color='black',label='Frontera de decision')
plt.legend(loc='upper left',framealpha=0.6)

Una alternativa es usar Principal Component analysis (PCA) para bajar de 12 a 2

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_este_pca=pca.fit_transform(X_este)
X_este_pca.shape

In [ ]:
plt.hist(X_este_pca[:,0],bins=10)
plt.xlabel(r'PCA 1')
plt.show()
plt.hist(X_este_pca[:,1],bins=10)
plt.xlabel(r'PCA 2')
plt.show()
plt.scatter(X_este_pca[y_este==0,0],X_este_pca[y_este==0,1], c='orange', label="No Playoff Este")
plt.scatter(X_este_pca[y_este==1,0],X_este_pca[y_este==1,1], c='green', label="Playoff Oeste")
plt.legend(loc='upper left',framealpha =0.1)
#plt.xlim(9.0,31.0)
#plt.ylim(15.0,35.0)
plt.xlabel(r'PCA 1')
plt.ylabel('PCA 2')
plt.show()

In [ ]:
LDA_este_pca=LinearDiscriminantAnalysis(solver='eigen')
LDA_este_pca.fit(X_este_pca,y_este)

In [ ]:
x=np.linspace(-25.0,25.0,100)
y=np.linspace(-15.0,15.0,100)
Xtoplot,Ytoplot=np.meshgrid(x,y)
plt.xlim(-25.0,25.0)
plt.ylim(-15.0,15.0)
Z=LDA_este_pca.predict(np.c_[Xtoplot.ravel(), Ytoplot.ravel()]).reshape(Xtoplot.shape)
plt.contourf(Xtoplot,Ytoplot,Z,levels=[0.0,0.5,1.0],colors=['orange','green'],alpha=0.6)
plt.scatter(X_este_pca[y_este==0,0],X_este_pca[y_este==0,1], c='orange', label="No Playoff Este")
plt.scatter(X_este_pca[y_este==1,0],X_este_pca[y_este==1,1], c='green', label="Playoff Oeste")
plt.legend(loc='upper left',framealpha =0.1)
plt.xlabel(r'PCA 1')
plt.ylabel('PCA 2')

In [ ]:
from sklearn.metrics import confusion_matrix
cf_este=confusion_matrix(y_este,LDA_este_pca.predict(X_este_pca))
print(cf_este)
tn, fp, fn, tp = cf_este.ravel()
print("Sensitividad (TPR/Recall) :%8.3f" % (tp/(tp+fn)))
print("Especificidad:%8.3f" % (tn/(tn+fp)))
print("Precision:%8.3f" % (tp/(tp+fp)))
print("FPR :%8.3f" % (fp/(tn+fp)))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax=MinMaxScaler()
print(LDA_este_pca.decision_function(X_este_pca).min(),LDA_este_pca.decision_function(X_este_pca).max())
decision=minmax.fit_transform(LDA_este_pca.decision_function(X_este_pca).reshape(-1,1))
print(decision.min(),decision.max())

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_este, LDA_este_pca.decision_function(X_este_pca))
print(thresholds[0],thresholds[-1])
plt.figure(figsize=(8, 6))                         # Not shown
plt.plot(fpr, tpr, linewidth=2, label=None)
plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal
plt.axis([0, 1, 0, 1])
plt.scatter(fpr[np.argmin(np.abs(thresholds-0.0))],tpr[np.argmin(np.abs(thresholds-0.0))],color='red')                                    # Not shown in the book
plt.xlabel('False Positive Rate (Fall-Out)', fontsize=16) # Not shown
plt.ylabel('True Positive Rate (Recall)', fontsize=16)    # Not shown
plt.grid(True)                                            # Not shown
plt.show()

# Test